# Query Suggestion Exploration

## Import Libraries

In [1]:
import pandas as pd

## Import query log data

In [10]:
files = ['data/Clean-Data-01.txt', 'data/Clean-Data-02.txt', 'data/Clean-Data-03.txt', 'data/Clean-Data-04.txt', 'data/Clean-Data-05.txt']

In [24]:
ql = pd.concat([pd.read_csv(f, sep='\t') for f in files])

In [26]:
ql = ql.sort_values(by=['AnonID', 'QueryTime'])

In [35]:
ql = ql.reset_index(drop=True)

In [42]:
len(ql)

3942354

In [75]:
ql[700000:700050]

,AnonID,Query,QueryTime
700000,1386368,southwest airlines,2006-03-06 20:00:48
700001,1386368,business number reversal,2006-03-07 10:46:52
700002,1386368,business phone numbers,2006-03-07 10:47:51
700003,1386368,las vegas show tickets,2006-03-08 21:39:16
700004,1386368,lance burton,2006-03-08 21:46:28
700005,1386368,san francisco bay burials,2006-03-08 21:52:31
700006,1386368,burial at sea and san francisco,2006-03-09 13:08:42
700007,1386368,monte carlo and las vegas,2006-03-09 18:41:26
700008,1386368,excalibur,2006-03-09 18:43:39
700009,1386368,new york new york hotel,2006-03-09 18:47:27
